# Data Download

In [78]:
from fredapi import Fred
import pandas as pd
import numpy as np
import statsmodels.api as sm

fred = Fred(api_key='3cca2c46ad523e6d1bb9269895f879fc')

billion = 1000000000
thousand = 1000
 
gdp = fred.get_series('GDPC96')*billion # Real GDP, Billions of Chained 2009 Dollars, Seasonally Adjusted Annual Rate
cons = fred.get_series('PCECC96')*billion # Real Personal Consumption Expenditures, Billions of Chained 2009 Dollars, Seasonally Adjusted Annual Rate
inv = fred.get_series('PNFI')*billion # Private Nonresidential Fixed Investment, Billions of Dollars, Seasonally Adjusted Annual Rate
domestic_income = fred.get_series('A261RX1Q020SBEA')*billion # Real gross domestic income, Billions of Chained 2009 Dollars, Seasonally Adjusted Annual Rate

pop = fred.get_series('B230RC0Q173SBEA')*thousand # Population (midperiod), Thousands, Seasonally Adjusted Annual Rate

non_farm_payroll = fred.get_series('PAYEMS')*thousand # All Employees: Total Nonfarm Payrolls, Thousands of Persons, Seasonally Adjusted
hours_index = fred.get_series("HOANBS") # Nonfarm Business Sector: Hours of All Persons, Index 2009=100, Seasonally Adjusted
average_weekly_hours = fred.get_series("AWHAETP") # Average Weekly Hours of All Employees: Total Private, Monthly

i = fred.get_series('TB3MS') # 3-Month Treasury Bill: Secondary Market Rate, Monthly
cpi = fred.get_series('CPIAUCSL') # Consumer Price Index for All Urban Consumers: All Items, Index 1982-1984=100, Seasonally Adjusted

# Data Processing

In [160]:
def data_processor(data, operation, freq):
    df = pd.DataFrame(np.array(data), index=pd.PeriodIndex(data.index, freq=freq), columns =[str(data)])
    if operation == "mean":
        df = df.groupby(pd.PeriodIndex(data.index, freq=freq), axis=0).mean().to_timestamp().loc["1947-01-01":"2017-01-01"]
    if operation == "sum":
        df = df.groupby(pd.PeriodIndex(data.index, freq=freq), axis=0).sum().to_timestamp().loc["1947-01-01":"2017-01-01"]
    return pd.Series(df[str(data)])

def table_1_replicator(data):
    first_order_autocorr = data.apply(lambda col: col.autocorr(lag=1), axis=0)
    corr = cycle.corr()["Y"]
    df = pd.concat([np.std(data)*100, np.std(data)/np.std(data)["Y"], first_order_autocorr, corr], axis=1)
    df.columns = ["Standard Deviation", "Relative Standard Deviation", "First Order Auto-correlation", "Contemporaneous Correlation with Output"]
    return df 

# Processing the nonfarm payroll time series
payroll = data_processor(non_farm_payroll, "mean", "Q")

# Processing the interest rate series
i_Q = data_processor(i, "mean", "Q")

# Processing the cpi series
cpi_Q = data_processor(cpi, "mean", "Q").pct_change().fillna(0)

# Processing the average weekly hours time series
hours_2009 = float(data_processor(average_weekly_hours["2009-01-01":"2009-12-31"], "mean", "A")*52/4)

# Quarterly Hours = Hours Index(index 2009=100) * Average Quarterly Hours (in 2009) / 100
q_hours = hours_index * hours_2009 / 100

# Average Real Wage = Real Income / (Average Number of Hours Worked * Number of Workers)
w = domestic_income / (q_hours*payroll)

# Real Interest Rate = Nominal Interest Rate - Inflation
r = i_Q - cpi_Q

# Creating the data frame
data = pd.concat([gdp/pop,cons/pop,inv["1947-01-01":"2017-01-01"]/pop,q_hours,(gdp/pop)/q_hours,w,r/100], axis=1)
data.columns = ["Y","C","I","N","Y/N","w","r"]

# Taking logs
data[["Y","C","I","N","Y/N","w"]] = np.log(data[["Y","C","I","N", "Y/N","w"]])

# Detrending with HP filter
cycle, trend = sm.tsa.filters.hpfilter(data, lamb=1600)

# Computing the first order detrenting
diff = data.diff(periods=1, axis=0).fillna(0)

# Results

In [162]:
# Replication of Table 1
table_1 = table_1_replicator(cycle)
table_1

,Standard Deviation,Relative Standard Deviation,First Order Auto-correlation,Contemporaneous Correlation with Output
Y,1.622407,1.000000,0.852104,1.000000
C,1.256600,0.774528,0.817933,0.774146
I,4.751245,2.928516,0.884540,0.674636
N,1.885469,1.162143,0.906565,0.871897
Y/N,0.923545,0.569244,0.762662,-0.023309
w,1.981371,1.221254,0.891902,-0.610364
r,1.087215,0.670125,0.825617,0.397839


In [163]:
# Replication of Table 1 with first order difference (without HP Filter)
table_1_diff = table_1_replicator(diff)
table_1_diff

,Standard Deviation,Relative Standard Deviation,First Order Auto-correlation,Contemporaneous Correlation with Output
Y,0.937155,1.000000,0.370393,1.000000
C,0.810694,0.865058,0.087621,0.774146
I,2.496455,2.663865,0.500531,0.674636
N,0.887600,0.947122,0.634116,0.871897
Y/N,0.666865,0.711584,0.058167,-0.023309
w,1.009620,1.077324,0.501164,-0.610364
r,0.667475,0.712236,0.235717,0.397839
